In [1]:
from glob import glob
from tqdm import tqdm
import json
import os
import pandas as pd
import numpy as np
import pickle
import sys
import cv2
from tqdm import tqdm

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model

In [6]:
img_height, img_width = 224, 224  # 이미지 크기
batch_size = 16  # batch size 너무 크게하면 메모리 초과 뜸

In [7]:
data_loc = 'C:/Users/multicampus/projects/fresh/data'  # your data loc

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f'{data_loc}/Training/images',
    validation_split=0.2,
    subset="training",
    seed=123,  # random 값 고정
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 139067 files belonging to 18 classes.
Using 111254 files for training.


In [9]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  f'{data_loc}/Training/images',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 139067 files belonging to 18 classes.
Using 27813 files for validation.


In [10]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f'{data_loc}/Validation/images',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 20098 files belonging to 18 classes.


In [11]:
cls = train_ds.class_names
num_classes = len(cls)

In [12]:
# def process(image,label):
#     image = tf.cast(image/255. ,tf.float32)
#     return image,label

# train_ds = train_ds.map(process)
# val_ds = val_ds.map(process)
# test_ds = test_ds.map(process)                   

In [13]:
# f = open('./my_model_class.txt', 'w')
# f.write(str(train_ds.class_names))
# f.close()

In [11]:
model = tf.keras.Sequential([
#   tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

# tf.keras.layers.Input(shape=(None, None, 3)),
# tf.keras.layers.Resizing(500, 500),

In [12]:
# model = tf.keras.Sequential([
# #   tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
#   tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(num_classes, activation='softmax')
# ])

In [13]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

In [14]:
# model.summary()

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

In [13]:
model.save('my_model_kind_1108.h5')

In [17]:
model.save('my_model_kind_1109.h5')

## mobile net

In [14]:
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
pre_trained_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [15]:
for layer in pre_trained_model.layers:
    layer.trainable = True

In [16]:
pre_trained_model.output

<KerasTensor: shape=(None, 7, 7, 1024) dtype=float32 (created by layer 'conv_pw_13_relu')>

In [17]:
def add_layer(model, num_classes):
    # 1 dimension의 output으로 flatten하기
    last_output = model.output
    x = tf.keras.layers.Flatten()(last_output)
    # 1,024개의 hidden unit과 ReLU activation을 포함한 fully connected layer 추가
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    # classification을 위한 마지막 sigmoid layer 추가하기
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = Model(model.input, x)
    return model

In [18]:
model = add_layer(pre_trained_model, num_classes)

In [19]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

In [20]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

Epoch 1/5
6954/6954 [==============================] - 1453s 208ms/step - loss: 0.2341 - accuracy: 0.9375 - val_loss: 0.1689 - val_accuracy: 0.9590
Epoch 2/5
6954/6954 [==============================] - 1523s 219ms/step - loss: 0.0888 - accuracy: 0.9735 - val_loss: 0.0437 - val_accuracy: 0.9848
Epoch 3/5
6954/6954 [==============================] - 1549s 223ms/step - loss: 0.0652 - accuracy: 0.9820 - val_loss: 0.0179 - val_accuracy: 0.9945
Epoch 4/5
6954/6954 [==============================] - 1389s 200ms/step - loss: 0.0480 - accuracy: 0.9867 - val_loss: 0.0169 - val_accuracy: 0.9942
Epoch 5/5
6954/6954 [==============================] - 1223s 176ms/step - loss: 0.0449 - accuracy: 0.9882 - val_loss: 0.0101 - val_accuracy: 0.9969


In [21]:
model.save('mobile_net_1111.h5')

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

In [ ]:
model.save('mobile_net_1112.h5')

## Test set으로 check

In [77]:
model = tf.keras.models.load_model("my_model_kind_1108.h5")

In [14]:
pred = []
y = []

In [17]:
for b_x, b_y in tqdm(test_ds):
    pred.extend(model.predict(b_x).tolist())
    y.extend(b_y.numpy().tolist())

100%|██████████| 1257/1257 [08:42<00:00,  2.41it/s]


In [18]:
pred = np.array(pred)
y = np.array(y)

In [19]:
pred_one = np.array([np.argmax(x) for x in pred])

In [20]:
np.mean(pred_one == y)

0.9230769230769231

## 사진 데이터 test

In [26]:
def img_preprocess(img_loc):
    img = cv2.imread(img_loc)
    img = img[:,:,::-1]  # bgr to rgb
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
#     img = img / 255
    img = img.reshape(1, 224, 224, 3)
    return img

In [58]:
img_loc = 'C:/Users/multicampus/projects/fresh/ap3.jpg'

In [59]:
img = img_preprocess(img_loc)

In [60]:
p = model.predict(img)

In [61]:
print(cls[p[0].argmax()])

apple_yanggwang


In [62]:
pd.DataFrame(p[0], cls).sort_values(0, ascending = False)

,0
apple_yanggwang,6.497335e-01
apple_fuji,2.159170e-01
onion_red,4.401521e-02
persimmon_bansi,1.507500e-02
mandarine_hallabong,1.283997e-02
onion_white,1.190384e-02
pear_singo,9.025316e-03
potato_sumi,8.397702e-03
persimmon_daebong,6.385315e-03
mandarine_onjumilgam,5.824568e-03


In [40]:
p

array([[1.3980662e-10, 2.1340211e-05, 7.6893708e-12, 4.0260730e-28,
        7.8801871e-17, 5.9421174e-29, 2.4851473e-12, 3.4651566e-05,
        5.7651682e-21, 2.0160280e-10, 1.6331439e-24, 4.5502165e-20,
        6.7753440e-01, 1.0686379e-02, 3.1172329e-01, 1.6961783e-27,
        1.5388557e-25, 5.5638545e-13]], dtype=float32)